# How to set up ROMS/CROCO simulation

Here we will see how to set up ROMS/CROCO files and time for Pyticles. 

Method is based using Modules/R_files.py Classes files and load.

___
## Setting up files

To set up your files you must chose a simulation name and add a switch in 
files Class where you define all files parameters. Here is an example of a
croco simulation named `polgyr`

```Python
# in R_files.py in files()
elif 'polgyr' in simul:
    
    '''
    time 00000 is 2001-01-10 14:53:20
    time 06511 is 2009-12-10 02:53:20
    
    conversion to uncompressed for 2006 = from 3620 to ...
    '''
    
    self.realyear = True
    self.realyear_origin = datetime(1999,1,1)
    self.model = 'croco'
    self.digits = 5

    folder= '/home/datawork-lops-osi/mlecorre/POLGYR/HIS/'
    self.grd='/home/datawork-lops-osi/mlecorre/POLGYR/INIT/polgyr_grd.nc'

    self.his=folder +'polgyr_his.'

    self.frc=folder + '/polgyr_his.03360.nc'
    self.wind=folder + '/polgyr_his.03360.nc'
    self.tfile=20
    self.tstart=0
    self.tend=1000

```
    
Configuration name: 'polgyr'

### model

model used to produce file. It is crucial to determine file names and
and to update file time stepping

- **croco/ucla:** files like `'polgyr_his_03660.nc'`. Note that number of digits can
    be defined by user

- **croco_clim:** CROCO naming for climatology runs with fixed number of
  outputs per file (usually 30 per month). `roms_his_Y2000M1.nc`

- **croco_inter:** CROCO naming for interannual runs with varying number of
  days per month. Also `roms_his_Y2000M1.nc` but 31 days in this one.

- **croco_xios** CROCO naming with XIOS CPP key:  `croco_1999-01-25-1999-01-29.nc`

### file naming and location

- **his** is used as a suffix for file naming

- **grid** should point to grid file (or file with grid parameters)

- **frc** and **wind** are optionnal

- **fileformat**: prefix of the file `.nc` most likely

### metadata parameters

- **realyear** is True for real calendar, otherwise 360 days and 30 days per month

- **realyear_origin**: datetime used for t=0 in croco simulation.
  This is also the date used to load at __init__ method of load()
  Class from R_files.py

- **realyear_tstart**: date that matched t_start=0 for Pyticles.
  This is very usefull when spinup is removed on disk to save some space.
  Typicall case is to remove a few years, then origin date and start date
  are different. 

- **digits**: number of digits to name files (2: 00, 3: 000, ...)

- **dtfile**: time step of ROMS/CROCO file in seconds

- **tfile** number of time steps per file (when fixed)

- **tstart**: time step within CROCO/ROMS file to start from. Usually it is
  set to zero, but it is possible to start from any time step between 0 and
  the number of time steps within the first file 

- **tend**: last time of simulation, take it big.

___
## practical case

It is a good practise before running to Pyticles to ensure that files are
correctly named and that time stepping is alright.


In [11]:
import sys
import numpy as np
from importlib import reload

# pyticles related modules and tools
sys.path.append("../../Modules/")
import R_files
from R_files import load
reload(R_files)
from R_files import load

here we load CROCO files from Polgyr simulation (available on Datarmor)

output logs should give you some information about simulation files and variables.

In [2]:
my_simul = 'polgyr'
simul = load(my_simul, touchfile=False)

simulname is polgyr
args (['polgyr'],)
args[0] ['polgyr']
len(args[0]) 1
time of simulation is: 0
/scratch/Jcollin/croco/polgyr/polgyr_his.00000.nc
not touching _his file, loading _grd anyway
[0, -1, 0, -1, [0]]
ncname0,ny1,ny2,nx1,nx2
/scratch/Jcollin/croco/polgyr/polgyr_grd.nc 0 -1 0 -1
[topo,pm,pn,f,lat,lon] have just been loaded
----------------------------------------------------------
All arrays are now Fortran ordered and indices are [i,j,k]
----------------------------------------------------------
(2002, 1602)


To check that time stepping is alright

- time 00000 is 2001-01-10 14:53:20

- and time 06511 is 2009-12-10 02:53:20

Using load method, and filetime, infiletime attributes

**NOTE** This only works when file is on disk

In [3]:
simul.update(time=6511)

# currently loaded date

try:
    print(f"{simul.date=}")
    # current year
    print(f"{simul.year=}")
    # time step in file
    print(f"{simul.infiletime=}")
    
except:
    print("file not in disk ?")

time of simulation is: 6511
/scratch/Jcollin/croco/polgyr/polgyr_his.06500.nc
no oceantime in file
file not in disk ?


In [22]:
my_simul = 'megatl3'
try:
    simul = load(simul=my_simul, touchfile=False)
except:
    pass

simul.update(time=100)

print(simul.ncname.model)

simulname is None
args (['megatl3'],)
args[0] ['megatl3']
len(args[0]) 1
time of simulation is: 0
/net/libra/local/tmp/1//gula/ROMS/Simulations/MEGATL/MEGATL3/MEGATL3_v2/megatl3_his.00000.nc
not touching _his file, loading _grd anyway
[0, -1, 0, -1, [0]]
time of simulation is: 100
/scratch/Jcollin/croco/polgyr/polgyr_his.00100.nc
no oceantime in file
croco


AttributeError: 'files' object has no attribute 'dtfile'

In [23]:
# croco_lionel ? 
my_simul = 'natl_GS'
try:
    simul = load(simul=my_simul, touchfile=False)
except:
    pass

simul.update(time=100)

print(simul.ncname.model)

simulname is None
args (['natl_GS'],)
args[0] ['natl_GS']
len(args[0]) 1
file is  /data0/project/meddle/gula/ROMS/Simulations/GS//all/natl_his_Y2000M01.0862.nc
/data0/project/meddle/gula/ROMS/Simulations/GS//all/natl_his_Y2000M01.0862.nc
not touching _his file, loading _grd anyway
[0, -1, 0, -1, [0]]
time of simulation is: 100
/scratch/Jcollin/croco/polgyr/polgyr_his.00100.nc
no oceantime in file
croco


In [2]:
# agrif climatology
my_simul = 'aacc_8k'

try:
    simul = load(simul=my_simul, touchfile=False)
except:
    pass

simul.update(time=100)

print(simul.ncname.model)

simulname is None
args (['aacc_8k'],)
args[0] ['aacc_8k']
len(args[0]) 1
/net/octant/local/tmp/1/collin/Gpla_8k/roms_his_Y20M1.nc
not touching _his file, loading _grd anyway
[0, -1, 0, -1, [0]]


NameError: name 'simul' is not defined

In [31]:
"""
Example with AGRIF model. 
Need to set up self.model = 'agrif' in Class files from R_files  

- a) climatology files (self.realyear = False)
    roms_his_Y2000M1.nc
    with 30 days per month and 12 months per year

- b) interannual files (self.realyear = True)

"""

reload(R_files)
from R_files import load

my_simul = 'aacc_8k'
simul = load(simul=my_simul, touchfile=False, loadgrid=False)

simul.update(time=101)

print(simul.ncfile)
print(simul.infiletime)


simulname is None
args (['aacc_8k'],)
args[0] ['aacc_8k']
len(args[0]) 1
/net/octant/local/tmp/1/collin/Gpla_8k/roms_his_Y20M1.nc
not touching _his file, loading _grd anyway
file is  /net/octant/local/tmp/1/collin/Gpla_8k/roms_his_Y20M11.nc
/net/octant/local/tmp/1/collin/Gpla_8k/roms_his_Y20M11.nc
no oceantime in file
/net/octant/local/tmp/1/collin/Gpla_8k/roms_his_Y20M11.nc
1
